In [130]:
import numpy as np
import damask
import yaml
import shutil
import os
from math import * 
os.getcwd()
print("The current directory is: ")
print(os.getcwd())

The current directory is: 
c:\Users\springnuance\Desktop\DAMASK3-Processing-Project


In [131]:
CPLaw = "PH"
dir = "convert_config_to_yaml"
inputFile = "material.config"
outputFile = "material.yaml"
inputDir = f"{dir}/{inputFile}"
outputDir = f"{dir}/{outputFile}"

PHparams_convert = np.load("convert_dicts/PHparams_convert.npy", allow_pickle=True).tolist()
DBparams_convert = np.load("convert_dicts/DBparams_convert.npy", allow_pickle=True).tolist()
output_convert = np.load("convert_dicts/output_convert.npy", allow_pickle=True).tolist()
lattice_convert = np.load("convert_dicts/lattice_convert.npy", allow_pickle=True).tolist()
params_convert = PHparams_convert if CPLaw == "PH" else DBparams_convert

def is_float(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

def convert_to_float(string):
    if is_float(string):
        return float(string)
    else:
        return string

## Cut the material.config file into 5 sections

In [132]:
with open(inputDir, "r") as file:
    content = file.read()
    #print(content)

# Extract the text under each header
headers = ["homogenization", "crystallite", "phase", "microstructure", "texture"]
headersBrackets = [f"<{header}>" for header in headers]

sections = content.split("#-------------------#")
headerIndex = []
for header in headersBrackets:
    for index, splitParts in enumerate(sections):
        # Check if stripPart contains the header
        if header in splitParts:
            headerIndex.append(index)
            break

sectionIndex = [index + 1 for index in headerIndex]
sectionsFiltered = [sections[index] for index in sectionIndex]


for header, section in zip(headers, sectionsFiltered):
    #print(section.split("\n"))
    # Remove empty lines from string section
    section = "\n".join([line for line in section.split("\n") if line.strip()])
    #print(section)
    outputFilePath = f"{dir}/{header}.txt"

    with open(outputFilePath, "w") as outputFile:
        outputFile.write(section)

    print(f"Saved {header}.txt")

print("Extraction and saving complete.")

data = {}

Saved homogenization.txt
Saved crystallite.txt
Saved phase.txt
Saved microstructure.txt
Saved texture.txt
Extraction and saving complete.


## Parsing homogenization

In [133]:
def parsingHomogenization(data):
    data["homogenization"] = {}

    with open(f"{dir}/homogenization.txt", "r") as file:
        for line in file:
            line = line.strip()
            if line.startswith("["):
                props = line.strip("[]")
                data["homogenization"][props] = {}
            elif line:
                key, value = line.split()
                data["homogenization"][props][key] = convert_to_float(value)
    
    # with open(outputDir, "a") as file:
    #     yaml.dump(data, file)
    #     #file.write("")

parsingHomogenization(data)
print("Homogenization parsed successfully")

Homogenization parsed successfully


## Parsing phase

In [134]:
# It is hard to parse the crystallite, so it is better you type the output here:

outputs = ["F", "P", "F_p", "F_e", "L_p", "O", "rho_mob", "rho_dip"] 
# "xi_sl", "gamma_sl"

elasticConstants = ["c11", "c12", "c44"]
interaction_coefficients = ["interaction_slipslip", "interaction_sliptwin", "interaction_twinslip", "interaction_twintwin"]

    
def parsingPhase(data):
    data['phase'] = {}

    with open(f"{dir}/phase.txt", "r") as file:
        for line in file:
            line = line.strip()
            if line.startswith("["):
                phaseName = line.strip("[]")
                data['phase'][phaseName] = {}
                data['phase'][phaseName]['lattice'] = {}
                data['phase'][phaseName]['mechanical'] = {}
                data['phase'][phaseName]['mechanical']['output'] = [i for i in outputs]
                data['phase'][phaseName]['mechanical']['elastic'] = {}
                data['phase'][phaseName]['mechanical']['plastic'] = {}
                #data['phase'][phaseName]['mechanical']['plastic']['output'] =  ["rho_mob", "rho_dip"]
            elif line:
                tokens = line.split()
                key, value = tokens[0], tokens[1]
                #print(key)
                if key == "lattice_structure": # DAMASK 2
                    data['phase'][phaseName]['lattice'] = lattice_convert[value]
                elif key == "elasticity": # DAMASK 2
                    data['phase'][phaseName]['mechanical']['elastic']['type'] = value.capitalize()
                elif key == "plasticity": # DAMASK 2
                    data['phase'][phaseName]['mechanical']['plastic']['type'] = value
                elif key in elasticConstants:
                    data['phase'][phaseName]['mechanical']['elastic'][params_convert[key]] = convert_to_float(value)
                elif key in interaction_coefficients:
                    data['phase'][phaseName]['mechanical']['plastic'][params_convert[key]] = [convert_to_float(i) for i in tokens[1:]]
                else:
                    data['phase'][phaseName]['mechanical']['plastic'][params_convert[key]] = convert_to_float(value)
    
    #print(data)
    
    # with open(outputDir, "a") as file:
    #     yaml.dump(data, file, default_flow_style=None, sort_keys=False)
    #     file.write("")

parsingPhase(data)
print("Phase parsed successfully")

Phase parsed successfully


In [135]:
# Load YAML file by DAMASK

materialConfig = damask.ConfigMaterial(data)
materialConfig.save(outputDir, default_flow_style=None, sort_keys=False)

## Parse the euler angles from texture.txt from Dream3D

In [136]:
def read_file_to_sentences(file_path):
    with open(file_path, "r") as file:
        content = file.read()
        sentences = content.split("\n")
        return sentences

def filteringSentences_angles(sentences):
    indices = [2,4,6]
    filteredSentences = [sentence.split() for i, sentence in enumerate(sentences)]
    #print(filteredSentences)
    filteredSentences = [[value for i, value in enumerate(sentence) if i in indices] for sentence in filteredSentences]
    # Removing empty lists which corresponds to empty lines
    filteredSentences = [sentence for sentence in filteredSentences if len(sentence) != 0]
    return filteredSentences

sentences = read_file_to_sentences(f"{dir}/texture.txt")
#print(sentences)
euler_angles = filteringSentences_angles(sentences)
print(euler_angles) # list of lists of 3 elements (phi1, Phi, phi2)
print("This should be equal to the number of grains in the microstructure")
print(len(euler_angles))

[['305.44', '67.67', '109.59'], ['30.00', '45.00', '45.00'], ['0.00', '0.00', '0.00'], ['183.76', '145.81', '309.79'], ['91.56', '121.94', '159.42'], ['143.76', '85.52', '1.19'], ['333.04', '84.81', '358.28'], ['87.51', '66.79', '62.95'], ['50.26', '140.36', '38.14'], ['304.77', '161.76', '189.44'], ['278.86', '41.29', '193.57'], ['13.69', '146.10', '234.90'], ['148.54', '116.46', '47.09'], ['139.16', '122.15', '323.65'], ['171.34', '108.26', '70.31'], ['75.93', '166.89', '209.11'], ['239.88', '68.06', '181.43'], ['302.64', '103.94', '266.17'], ['8.57', '114.37', '82.59'], ['85.44', '64.71', '228.11'], ['90.00', '54.70', '45.00'], ['158.53', '126.68', '310.40'], ['314.62', '86.39', '144.87'], ['244.57', '132.81', '228.45'], ['303.47', '151.75', '353.09'], ['30.00', '45.00', '45.00'], ['216.32', '55.26', '241.16'], ['120.11', '65.55', '26.75'], ['101.02', '116.69', '45.49'], ['16.29', '49.25', '142.99'], ['286.18', '100.52', '240.16'], ['328.73', '145.35', '308.73'], ['251.13', '43.92',

## Parse the phase ID from microstructure.txt from Dream3D

In [137]:
def read_file_to_sentences(file_path):
    with open(file_path, "r") as file:
        content = file.read()
        sentences = content.split("\n")
        return sentences

def filteringSentences_phase(sentences):
    #print(sentences)
    indices = [2]
    filteredSentences = [sentence.split() for i, sentence in enumerate(sentences)]
    #print(filteredSentences)
    #print(filteredSentences)
    filteredSentences = [[value for i, value in enumerate(sentence) if i in indices] for sentence in filteredSentences]
    #print(filteredSentences)
    # Removing empty lists which corresponds to empty lines
    phase_indices = [sentence for sentence in filteredSentences if len(sentence) != 0]
    phase_indices = [int(i[0]) for i in phase_indices]
    return phase_indices

phases = list(data["phase"].keys())
homogenization = list(data["homogenization"].keys())[0]
sentences = read_file_to_sentences(f"{dir}/microstructure.txt")
#print(sentences)
phase_indices = filteringSentences_phase(sentences)
phase_texture = [phases[i-1] for i in phase_indices]
print(phase_texture) 
print("This should be equal to the number of grains in the microstructure")
print(len(phase_texture))

['BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Ferrite', 'BCC_Martensite', 'BCC_Martensite', 'BCC_Martensite', 'BCC_Martensite', 'BCC_Martensite', 'BCC_Martensite', 'BCC_Martensite', 'BCC_Martensite', 'BCC_Martensite', 'BCC_Martensite', 'BCC_Martensite', 'BCC_Martensite', 'BCC_Martensite', 

In [138]:
materialConfig = materialConfig.material_add(
    #O=damask.Rotation.from_random(2),
    # degrees : bool, optional. Euler angles are given in degrees. Defaults to False which is in radian
    O=damask.Rotation.from_Euler_angles(np.array(euler_angles), degrees=True),
    phase=phase_texture,
    homogenization=homogenization)

In [139]:
# Saving material.yaml file in convert_config_to_yaml folder
materialConfig.save(outputDir, default_flow_style=None, sort_keys=False)